In [1]:
%load_ext lab_black

In [2]:
import random
from pathlib import Path

import numpy as np
import open3d as o3d
import torch as th
import yaml
from easydict import EasyDict
from kitsu.utils import instantiate_from_config
from kitsu.utils.vis3d import save_mesh

from src.ditto.generate import Generator3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
th.set_grad_enabled(False)

# ShapeNet Reconstruction

## Load Options

In [6]:
args_path = Path("results/ditto/shapenet/00/args.yaml")
ckpt_path = sorted(list(args_path.parent.glob("best*.pth")))[-1]
ckpt_path

PosixPath('results/ditto/shapenet/00/best.pth')

In [7]:
with open(args_path) as f:
    args = EasyDict(yaml.safe_load(f))

## Load Dataset

In [8]:
args.dataset

{'target': 'src.ditto.dataset.dataset.load_dataloaders',
 'params': {'batch_size': 32, 'num_workers': 8}}

In [ ]:
# If you already downloaded dataset
# dl_train, dl_valid, dl_test = instantiate_from_config(args.dataset, cfg=args.cfg)

# idx = random.randint(0, len(dl_test.dataset) - 1)
# data = dl_test.collate_fn([dl_test.dataset[1000]])

In [9]:
# If you don't have dataset yet
data = np.load("assets/demo_data.npz", allow_pickle=True)
data = {k: th.from_numpy(v) for k, v in data.items()}

## Load Model and Checkpoint

In [10]:
args.model

{'target': 'src.ditto.ulto.ULTO',
 'params': {'encoder': {'target': 'src.ditto.encoder.FKAConvEncoder2d',
   'params': {'dim': 32,
    'dim_in': 3,
    'res': 64,
    'padding': 0.1,
    'n_blocks': 5}},
  'unet': {'target': 'src.ditto.unet2d.UNet2d',
   'params': {'dim': 32,
    'dim_in': 32,
    'dim_out': 32,
    'depth': 4,
    'act_fn': 'relu',
    'padding': 0.1,
    'window_size': 125,
    'head_dim': 32,
    'rodin_conv': True,
    'n_kernels': 1,
    'rotary_relative_pe': False}},
  'decoder': {'target': 'src.ditto.decoder.ULTODecoder2d',
   'params': {'dim': 32,
    'dim_out': 1,
    'n_blocks': 4,
    'padding': 0.1,
    'act_fn': 'relu',
    'num_neighbors': 32,
    'head_dim': 32}}}}

In [11]:
model = instantiate_from_config(args.model).cuda()

In [12]:
ckpt = th.load(ckpt_path, map_location="cpu")
model.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [13]:
# generator helps generating meshes from implicit model
cfg = args.cfg
generator = Generator3D(
    model,
    global_resolution=128,
    device="cuda",
    threshold=cfg["test"]["threshold"],
    resolution0=cfg["generation"]["resolution_0"],
    upsampling_steps=cfg["generation"]["upsampling_steps"],
    sample=cfg["generation"]["use_sampling"],
    refinement_step=cfg["generation"]["refinement_step"],
    simplify_nfaces=cfg["generation"]["simplify_nfaces"],
    input_type=cfg["data"]["input_type"],
    padding=cfg["data"]["padding"],
    vol_info=None,
    vol_bound=None,
)

## Reconstruct Meshes

In [15]:
mesh, stat = generator.generate_mesh(data)

In [16]:
o3d.io.write_triangle_mesh("test.obj", mesh)

True

## Load Options

In [5]:
args_path = Path("results/ditto/shapenet/00/args.yaml")
ckpt_path = sorted(list(args_path.parent.glob("best*.pth")))[-1]
ckpt_path

PosixPath('results/ditto/shapenet/00/best.pth')

In [6]:
with open(args_path) as f:
    args = EasyDict(yaml.safe_load(f))

## Load Dataset

In [7]:
args.dataset

{'target': 'src.ditto.dataset.dataset.load_dataloaders',
 'params': {'batch_size': 32, 'num_workers': 8}}

In [8]:
dl_train, dl_valid, dl_test = instantiate_from_config(args.dataset, cfg=args.cfg)
len(dl_train.dataset), len(dl_valid.dataset), len(dl_test.dataset)

(30661, 4371, 8751)

## Load Model and Checkpoint

In [9]:
args.model

{'target': 'src.ditto.ulto.ULTO',
 'params': {'encoder': {'target': 'src.ditto.encoder.FKAConvEncoder2d',
   'params': {'dim': 32,
    'dim_in': 3,
    'res': 64,
    'padding': 0.1,
    'n_blocks': 5}},
  'unet': {'target': 'src.ditto.unet2d.UNet2d',
   'params': {'dim': 32,
    'dim_in': 32,
    'dim_out': 32,
    'depth': 4,
    'act_fn': 'relu',
    'padding': 0.1,
    'window_size': 125,
    'head_dim': 32,
    'rodin_conv': True,
    'n_kernels': 1,
    'rotary_relative_pe': False}},
  'decoder': {'target': 'src.ditto.decoder.ULTODecoder2d',
   'params': {'dim': 32,
    'dim_out': 1,
    'n_blocks': 4,
    'padding': 0.1,
    'act_fn': 'relu',
    'num_neighbors': 32,
    'head_dim': 32}}}}

In [10]:
model = instantiate_from_config(args.model).cuda()

In [11]:
ckpt = th.load(ckpt_path, map_location="cpu")
model.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [12]:
# generator helps generating meshes from implicit model
cfg = args.cfg
generator = Generator3D(
    model,
    global_resolution=128,
    device="cuda",
    threshold=cfg["test"]["threshold"],
    resolution0=cfg["generation"]["resolution_0"],
    upsampling_steps=cfg["generation"]["upsampling_steps"],
    sample=cfg["generation"]["use_sampling"],
    refinement_step=cfg["generation"]["refinement_step"],
    simplify_nfaces=cfg["generation"]["simplify_nfaces"],
    input_type=cfg["data"]["input_type"],
    padding=cfg["data"]["padding"],
    vol_info=None,
    vol_bound=None,
)

## Reconstruct Meshes

In [17]:
idx = random.randint(0, len(dl_test.dataset) - 1)
data = dl_test.collate_fn([dl_test.dataset[1000]])

In [14]:
mesh, stat = generator.generate_mesh(data)

In [15]:
o3d.io.write_triangle_mesh("test.obj", mesh)

True

In [ ]:
# You can now visualize the output mesh 'test.obj' using rendering tools like MeshLab.

In [18]:
data

{'points': tensor([[[-0.4934, -0.4250,  0.2910],
          [-0.4797, -0.0347, -0.4546],
          [-0.5277, -0.2440, -0.1853],
          ...,
          [ 0.0437,  0.4869, -0.3456],
          [ 0.2282, -0.3981,  0.1256],
          [ 0.1138,  0.0566,  0.1275]]]),
 'points.occ': tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 'points.sub_points_idx': tensor([[58778, 61155, 20685,  ..., 28697, 70745, 60644]]),
 'points_iou': tensor([[[ 0.4318, -0.5246,  0.0220],
          [-0.3407,  0.5053,  0.0417],
          [-0.3310, -0.4791,  0.2901],
          ...,
          [-0.4737, -0.2209,  0.0633],
          [-0.2053, -0.5097, -0.1222],
          [ 0.3800,  0.3899, -0.1080]]]),
 'points_iou.occ': tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 'inputs': tensor([[[ 0.1418,  0.2025,  0.3714],
          [ 0.1640, -0.2871,  0.0832],
          [ 0.1205, -0.2376, -0.4999],
          ...,
          [ 0.2784, -0.0760,  0.1367],
          [ 0.0398, -0.0125,  0.3809],
          [-0.0430, -0.0058, -0.2017]]]),
 'inpu

In [21]:
data_np = {k: v.numpy() for k, v in data.items()}

In [22]:
np.savez_compressed("assets/demo_data.npz", **data_np)